## Load the LWF Dataset

In [ ]:
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt

# Download the LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# Access data
images = lfw_people.images
labels = lfw_people.target
target_names = lfw_people.target_names

# Showing the first 10 images
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(images[i], cmap='gray')
    ax.set_title(target_names[labels[i]])
    ax.axis('off')

plt.show()


## Save the images,labes and names

In [ ]:
import pickle


# Save the data to a file
with open('Dataset\\lfw_people.pkl', 'wb') as f:
    pickle.dump(lfw_people, f)

# Save images, labels, and target names separately
images = lfw_people.images
labels = lfw_people.target
target_names = lfw_people.target_names

# Saving individual components in case we need them seperated
with open('Dataset\\lfw_images.pkl', 'wb') as f_img, open('Dataset\\lfw_labels.pkl', 'wb') as f_lbl, open('Dataset\\lfw_target_names.pkl', 'wb') as f_names:
    pickle.dump(images, f_img)
    pickle.dump(labels, f_lbl)
    pickle.dump(target_names, f_names)

print("LFW dataset saved successfully.")


## Using ResNet50 for embedding

## Storing the embeddings to FAISS vector db